In [1]:
# import libraries 
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [5]:
shapes = pd.read_csv("../businfo/shapes.txt")
stops = pd.read_csv("../businfo/stops.txt")

In [7]:
trips = pd.read_csv('../businfo/trips.txt')

In [9]:
routes = pd.read_csv('../businfo/routes.txt')

In [11]:
stop_times = pd.read_csv('../businfo/stop_times.txt')

In [12]:
print(stop_times)

                                    trip_id arrival_time departure_time  \
0               1286-79922-51600-2-a7688ca4     14:20:00       14:20:00   
1               1286-79922-51600-2-a7688ca4     14:25:45       14:25:45   
2               1286-79922-51600-2-a7688ca4     14:31:59       14:32:00   
3               1286-79922-51600-2-a7688ca4     14:34:15       14:34:15   
4               1286-79922-51600-2-a7688ca4     14:35:30       14:35:30   
...                                     ...          ...            ...   
828598  249-820232-52920-2-2217321-ccc91c9b     15:19:00       15:19:00   
828599  249-820232-52920-2-2217321-ccc91c9b     15:22:00       15:22:00   
828600  249-820232-52920-2-2217321-ccc91c9b     15:24:00       15:24:00   
828601  249-820232-52920-2-2217321-ccc91c9b     15:27:00       15:27:00   
828602  249-820232-52920-2-2217321-ccc91c9b     15:31:00       15:31:00   

              stop_id  stop_sequence stop_headsign  pickup_type  \
0       7107-14d27628           

In [14]:
print(trips.columns)

Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name',
       'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible',
       'bikes_allowed'],
      dtype='object')


In [15]:
print(stops)


             stop_id  stop_code                        stop_name  stop_desc  \
0       100-56c57897        100         Papatoetoe Train Station        NaN   
1      1002-adf82201       1002              Stop A Lower Albert        NaN   
2      1003-ea94d2b2       1003              Stop B Lower Albert        NaN   
3     10036-288916d9      10036                        Ellerslie        NaN   
4      1004-42737000       1004              Stop C Lower Albert        NaN   
...              ...        ...                              ...        ...   
6681   9925-fa8611dc       9925  Sam Wrigley Street/ Cook Street        NaN   
6682   9974-6b1dd4cc       9974                    Madison Place        NaN   
6683   9975-9f7b8faa       9975                   Linwood Avenue        NaN   
6684   9976-69afaf1c       9976                  Linwood  Avenue        NaN   
6685   9986-e705dd1d       9986                    Matakana Road        NaN   

      stop_lat   stop_lon  zone_id  stop_url  locat

# EXTRACTING STOP TABLE FOR SQL TABLE

In [23]:
stop_table = stops
stop_table['stop_coordinates'] = stop_table.apply(lambda row : [row['stop_lon'], row['stop_lat']], axis = 1)
stop_table = stop_table[['stop_id', 'stop_coordinates']]
print(stop_table)

             stop_id        stop_coordinates
0       100-56c57897  [174.84925, -36.97766]
1      1002-adf82201  [174.76575, -36.84326]
2      1003-ea94d2b2  [174.76551, -36.84381]
3     10036-288916d9  [174.80712, -36.89726]
4      1004-42737000   [174.7655, -36.84403]
...              ...                     ...
6681   9925-fa8611dc  [174.75679, -36.85073]
6682   9974-6b1dd4cc  [174.74654, -36.75943]
6683   9975-9f7b8faa  [174.74401, -36.75716]
6684   9976-69afaf1c  [174.74761, -36.75667]
6685   9986-e705dd1d  [174.66156, -36.39619]

[6686 rows x 2 columns]


In [25]:
from sqlalchemy import create_engine, Float, Integer, String, ARRAY

# Replace these with your PostgreSQL connection details
db_username = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "bus_trial"

# Create a database connection string
db_uri = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define the column data types
column_types = {
    'stop_id': String,
    'stop_coordinates': ARRAY(Float)
}

# Store the dataframe in the PostgreSQL database
stop_table.to_sql('stop_table', engine, if_exists='replace', dtype=column_types, index = False)


686

# EXTRACTING SHAPE STOPS (BUS NUMBER, SHAPE_ID, STOP_ID)

## Collecting the dataframe 

In [30]:
trips.join(stop_times, on = 'trip_id')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [47]:
stop_trip_data = pd.merge(trips, stop_times, how = "left", on = 'trip_id')
stop_trip_data = pd.merge(stop_trip_data, routes, how = 'left', on = 'route_id')

stop_data = stop_trip_data[['route_short_name', 'trip_id', 'shape_id', 'stop_id']]
stop_data.columns = ['route_name', 'trip_id', 'shape_id', 'stop_id']

distinct_data = stop_data[['route_name', 'shape_id', 'stop_id']].drop_duplicates()

In [46]:
stop_data

,route_name,trip_id,shape_id,stop_id
0,981,1010-98109-18000-2-fc6666b6,1010-98109-d221c68a,4791-27dc4f30
1,981,1010-98109-18000-2-fc6666b6,1010-98109-d221c68a,4778-c7bea67d
2,981,1010-98109-18000-2-fc6666b6,1010-98109-d221c68a,4782-73517bf7
3,981,1010-98109-18000-2-fc6666b6,1010-98109-d221c68a,4797-11232403
4,981,1010-98109-18000-2-fc6666b6,1010-98109-d221c68a,4799-3aa1c6f6
...,...,...,...,...
828598,126,957-12602-32400-2-ea1af35a,957-12602-45320708,3588-6006c6e9
828599,126,957-12602-32400-2-ea1af35a,957-12602-45320708,4315-eb4f0338
828600,126,957-12602-32400-2-ea1af35a,957-12602-45320708,4259-6945389a
828601,126,957-12602-32400-2-ea1af35a,957-12602-45320708,3744-2d944a90


## Insert into database

In [48]:
from sqlalchemy import create_engine, Float, Integer, String, ARRAY

# Replace these with your PostgreSQL connection details
db_username = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "bus_trial"

# Create a database connection string
db_uri = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define the column data types
column_types = {
    'route_name': String,
    'shape_id': String,
    'stop_id': String
}

# Store the dataframe in the PostgreSQL database
distinct_data.to_sql('stoptrips_table', engine, if_exists='replace', dtype=column_types, index = False)


210